In [6]:
def get_vmd_bondlist(pdb_file):
    with open (pdb_file) as f_:
        f_readlines = f_.readlines()
    conect_part = []
    for i in f_readlines:
        if 'CONECT' in i:
            conect_part.append(i)
    conect_part = sorted(conect_part, key=lambda x: int(x.split()[1]))
    vmd_bondlist = '{ '
    n_bond = 0
    for i in conect_part:
        tmp = map(str, set([int(j) - 1 for j in i.split()[2:]]))
        vmd_bondlist += "{" + ' '.join(tmp) + "} "
        n_bond += len(set(i.split()[2:]))
    vmd_bondlist += '}'
    return vmd_bondlist

def gen_lmp_data(pdb_file):
    import os
    vmd_bondlist = get_vmd_bondlist(pdb_file)
    pdb_name = ''.join(pdb_file.split('.')[:-1])
    tcl_cmd = f'''package require topotools;
topo clearbonds;
#mol bondsrecalc top;
atomselect list
set sel [atomselect 0 "all"]
$sel setbonds {vmd_bondlist}
#topo setbondlist {{1 2}}
topo guessangles;
topo guessdihedrals;
topo guessimpropers;
topo retypebonds;
topo retypeangles;
topo retypedihedrals;
mol reanalyze top;
topo writelammpsdata {pdb_name}-lmp.data;
quit;    
'''
    ret = os.popen(f"echo '{tcl_cmd}' | vmd -dispdev none {pdb_file}").read()
    #print (tcl_cmd, ret) ##for debug
    return None

In [27]:
if __name__ == '__main__':
    # import sys
    # pdb_file = sys.argv[1]
    pdb_file = '6-6-d-1.5-connect.pdb'
    gen_lmp_data(pdb_file)

/home/kwwan/software/vmd-1.9.4a51-path/vmd_home/vmd_LINUXAMD64: /lib64/libGL.so.1: no version information available (required by /home/kwwan/software/vmd-1.9.4a51-path/vmd_home/vmd_LINUXAMD64)
